In [1]:
!curl https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt -o data/The_Verdict.txt
import re
with open("data/The_Verdict.txt", "r") as f:
    raw_text = f.read()

raw_text = raw_text[98::]
raw_text = re.sub(r"(\s+)"," ", raw_text)
print("Number of characters {}", len(raw_text))
print(raw_text[:100])

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20479  100 20479    0     0  68995      0 --:--:-- --:--:-- --:--:-- 69185
Number of characters {} 20299
 great surprise to me to hear that, in the height of his glory, he had dropped his painting, married


In [2]:
# Naive splitting
text = "Hello, world. This, is a--test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a--test.']


In [3]:
text="Hello, world. Is this-- a test?"
result = re.split(r'([\(\),.\s;:>_!]|--)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test?']


In [4]:
def basic_tokenizer(input: str)->list[str]:
    result = re.split(r'([,.;:?_!""()\']|--|\s)', input)
    result = [item.strip() for item in result if item.strip()]
    return result

In [5]:
basic_tokenizer("this is a simple test?")

['this', 'is', 'a', 'simple', 'test', '?']

In [6]:
len(basic_tokenizer(raw_text))


4669

In [7]:
basic_tokenizer(raw_text)[0:30]

['great',
 'surprise',
 'to',
 'me',
 'to',
 'hear',
 'that',
 ',',
 'in',
 'the',
 'height',
 'of',
 'his',
 'glory',
 ',',
 'he',
 'had',
 'dropped',
 'his',
 'painting',
 ',',
 'married',
 'a',
 'rich',
 'widow',
 ',',
 'and',
 'established',
 'himself',
 'in']

In [8]:
preprocessed = basic_tokenizer(raw_text)
all_words = sorted(set(preprocessed))
len(all_words)

1127

In [9]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [10]:
vocab

{'!': 0,
 '"': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '--': 6,
 '.': 7,
 ':': 8,
 ';': 9,
 '?': 10,
 'A': 11,
 'Ah': 12,
 'Among': 13,
 'And': 14,
 'Are': 15,
 'Arrt': 16,
 'As': 17,
 'At': 18,
 'Be': 19,
 'Begin': 20,
 'Burlington': 21,
 'But': 22,
 'By': 23,
 'Carlo': 24,
 'Chicago': 25,
 'Claude': 26,
 'Come': 27,
 'Croft': 28,
 'Destroyed': 29,
 'Devonshire': 30,
 'Don': 31,
 'Dubarry': 32,
 'Emperors': 33,
 'Florence': 34,
 'For': 35,
 'Gallery': 36,
 'Gideon': 37,
 'Gisburn': 38,
 'Gisburns': 39,
 'Grafton': 40,
 'Greek': 41,
 'Grindle': 42,
 'Grindles': 43,
 'Had': 44,
 'Hang': 45,
 'Has': 46,
 'He': 47,
 'Her': 48,
 'Hermia': 49,
 'His': 50,
 'How': 51,
 'I': 52,
 'If': 53,
 'In': 54,
 'It': 55,
 'Jack': 56,
 'Jove': 57,
 'Just': 58,
 'Lord': 59,
 'Made': 60,
 'Miss': 61,
 'Money': 62,
 'Monte': 63,
 'Moon-dancers': 64,
 'Mr': 65,
 'Mrs': 66,
 'My': 67,
 'Never': 68,
 'No': 69,
 'Now': 70,
 'Nutley': 71,
 'Of': 72,
 'Oh': 73,
 'On': 74,
 'Once': 75,
 'Only': 76,
 'Or': 77,
 'P

In [11]:


class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {value:key for key,value in vocab.items()}

    def encode(self, text):
        result = re.split(r'([,.;:?_!""()\']|--|\s)', text)
        result = [item.strip() for item in result if item.strip()]
        tokens = [self.str_to_int[word] for word in result]
        return tokens

    def decode(self, ids):
        text = " ".join(self.int_to_str[token] for token in ids)
        text = re.sub(r'\s+([,.?!"()\'])'  , r'\1', text)
        return text
      

        

In [14]:
!uv pip install tiktoken
from importlib.metadata import version
import tiktoken
version("tiktoken")

Using Python 3.11.11 environment at: /Users/benoit/Projects/build-a-llm-exercices/.venv
Audited 1 package in 1ms


'0.12.0'

In [15]:
tokenizer = tiktoken.get_encoding("gpt2")

In [16]:
text = "Helllo - do you like tea ? <|endoftext|> randomUnknownWord yeah"
integer = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
integer

[28254,
 5439,
 532,
 466,
 345,
 588,
 8887,
 5633,
 220,
 50256,
 4738,
 20035,
 26449,
 10194]

In [17]:
with open("data/The_Verdict.txt", "r") as f:
    raw_text = f.read()

In [18]:
enc_text = tokenizer.encode(raw_text)

In [19]:
len(enc_text)

5145

In [20]:
enc_sample = enc_text[50:]

In [21]:
context_size = 4 # How many tokens in the input
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

In [22]:
print(x)
print(y)

[290, 4920, 2241, 287]
[4920, 2241, 287, 257]


In [23]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), '->', tokenizer.decode([desired]))

 and ->  established
 and established ->  himself
 and established himself ->  in
 and established himself in ->  a


In [24]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):

        self.input_ids = []
        self.target_ids = []
        
        token_ids = tokenizer.encode(txt)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

        print(len(self.input_ids), len(self.input_ids[0]))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]


def create_dataloader_v1(txt, 
                         batch_size = 4, 
                         max_length=256, 
                         stride=128, 
                         shuffle=True, 
                         drop_last=True, 
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset =  GPTDatasetV1(txt, 
                            tokenizer=tokenizer, 
                            max_length=max_length,
                            stride=stride) 
    return DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers # Number of CPU Processes to use, 
        # 0 means no multi-processing
    )
    

/Users/benoit/Projects/build-a-llm-exercices/.venv/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [25]:
## Creating dataloader with batch size 1 for LLM with context size of 4 

dataloader = create_dataloader_v1(
    txt=raw_text,
    batch_size=1,
    max_length=4,
    stride=1,
    shuffle=False,
    num_workers=0
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
first_batch

5141 4


[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]

In [26]:
len(raw_text)/4

5119.75

In [27]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [28]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4)

1286 4


In [29]:
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("Targets:\n", targets)

Inputs:
 tensor([[  673,   373, 10032,   286],
        [   30,  4162,    11,   314],
        [ 9074,    13,   520,  5493],
        [  465,  5986,  1165,    30],
        [32796,  2637,  3244,   465],
        [  314,  2900,    11,   616],
        [  645, 15223,   670,   286],
        [  286,   511,   512,  1741]])
Targets:
 tensor([[  373, 10032,   286,   852],
        [ 4162,    11,   314,   550],
        [   13,   520,  5493,   339],
        [ 5986,  1165,    30,   314],
        [ 2637,  3244,   465,  6283],
        [ 2900,    11,   616,  4151],
        [15223,   670,   286,  8166],
        [  511,   512,  1741,    13]])


#### Token Embeddings


In [30]:
vocab_size = 6 
output_dim = 3

In [31]:
torch.manual_seed(123) # Reproducibility
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)
print(embedding_layer(torch.tensor([3])))

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [32]:
input_ids = torch.tensor([2, 3, 5, 1])
embedding_layer(input_ids)

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)

In [33]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [34]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=8,
    max_length=max_length,
    stride=max_length,
    shuffle=False,
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Tokens IDs:\n", inputs)

1286 4
Tokens IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])


In [35]:
print("Inputs shape", inputs.shape)

Inputs shape torch.Size([8, 4])


In [36]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)


torch.Size([8, 4, 256])


### Data Batch Size
8 Samples of size 4 (=nb of Tokens) in a 256 Dimensional Vector